## Step - 2: Create network of subjects

In [1]:
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from peoplefinder import PeopleFinder
from operator import attrgetter
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

def subject_AandB(cur,A,B):
    #calculate the number of bills that have both subject A and subject B. Both are strings.
    #query for entires with both subjects.
    query = ("SELECT count(index) FROM bills WHERE category LIKE 'passage' AND (subjects LIKE '%%{" + A
                     + ",%%' OR subjects LIKE '%%,"+A+",%%' OR subjects LIKE '%%,"+A+"}%%') AND (subjects LIKE '%%{" + B 
                     +",%%' OR subjects LIKE '%%,"+B+",%%' OR subjects LIKE '%%,"+B+"}%%');")
    cur.execute(query)
    return ((cur.fetchall())[0][0])
    #get the size of the rows, i.e., the number of entries returned.
    #s = foo['index'].size
    #return s

def subject_AorB(engine,A,B):
    #calculate the number of bills that have either subject A and subject B. Both are strings.
    #query for entires with both subjects.
    #query1 = ("SELECT * FROM bills WHERE category LIKE 'passage' AND (subjects LIKE '%%{" + A
    #                 + ",%%' OR subjects LIKE '%%,"+A+",%%' OR subjects LIKE '%%,"+A+"}%%');")
    #query2 = ("SELECT * FROM bills WHERE category LIKE 'passage' AND (subjects LIKE '%%{" + A
    #                 + ",%%' OR subjects LIKE '%%,"+A+",%%' OR subjects LIKE '%%,"+A+"}%%');")
    #use query and put in DataFrame
    #foo = pd.read_sql_query(query1,engine)
    #get the size of the rows, i.e., the number of entries returned.
    #s1 = foo['index'].size
    #print(s1)
    #foo = pd.read_sql_query(query2,engine)
    #get the size of the rows, i.e., the number of entries returned.
    #s2 = foo['index'].size
    #print(s2)
    #s3 = subject_AandB(engine,A,B)
    #print(s3)
    result = s1 + s2 - s3
    return result

def subject_count(engine,A):
    query = ("SELECT * FROM bills WHERE category LIKE 'passage' AND (subjects LIKE '%%{" + A
                     + ",%%' OR subjects LIKE '%%,"+A+",%%' OR subjects LIKE '%%,"+A+"}%%');")
    #use query and put in DataFrame
    foo = pd.read_sql_query(query,engine)
    return (foo['index'].size)

def fast_subject_count(cur,A):
    query = ("SELECT count(index) FROM bills WHERE category LIKE 'passage' AND (subjects LIKE '%%{" + A
                     + ",%%' OR subjects LIKE '%%,"+A+",%%' OR subjects LIKE '%%,"+A+"}%%');")
    cur.execute(query)
    return ((cur.fetchall())[0][0])

def super_fast_subject_count(cur,A):
    query = ()
    return
    
def jaccard_similarity(cur,A,B,nA,nB):
    nAandB = subject_AandB(cur,A,B)
    result = nAandB/(nA+nB-nAandB)
    return result

def get_all_subjects(engine):
    query = "SELECT * FROM topics"
    foo = pd.read_sql_query(query, engine)
    return foo

def clean_subject(a):
    try:
        quoteloc = a.index("'") #may get issues with multiple single quotes in one topic.
        a = a[:quoteloc]+"'"+a[quoteloc:]
    except:
        pass #no quotes found
    #need to add " if there are any spaces in the subject
    try:
        quoteloc = a.index(" ") #will go to except if there are no spaces in the subject
        a = '\"'+a+'\"'
    except:
        pass #no spaces found
    return a

Populating the interactive namespace from numpy and matplotlib


In [2]:
#set up postgresql engine
dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)
conn = psycopg2.connect("dbname='legislatr' user='lordluen' host='localhost'")
cur = conn.cursor()

In [3]:
#get all subjects
subs = get_all_subjects(engine)
#change syntax to be correct for queries.
subs['q_subjects'] = subs['subjects'].map( lambda x: clean_subject(x))
#can probably get number of bills each subject is on and store it before the loop to speed it up a bit.
subs['numBills'] = subs['q_subjects'].map( lambda x: fast_subject_count(cur,x))
#delete subjects that are appear less than 10 times
subs = subs[subs.numBills >= 10]

In [4]:
subs.head(11)

,index,subjects,q_subjects,numBills
1,1,Abortion,Abortion,40
2,2,Academic performance and assessments,"""Academic performance and assessments""",17
3,3,Access Board,"""Access Board""",13
4,4,Accidents,Accidents,11
5,5,Accounting and auditing,"""Accounting and auditing""",73
7,7,Administrative Conference of the U.S.,"""Administrative Conference of the U.S.""",14
8,8,Administrative law and regulatory procedures,"""Administrative law and regulatory procedures""",284
9,9,Administrative remedies,"""Administrative remedies""",120
10,10,Adoption and foster care,"""Adoption and foster care""",14
13,13,Advanced technology and technological innovations,"""Advanced technology and technological innovat...",48


In [38]:
subs_size = subs['subjects'].size
print('subs_size = ',subs_size)
subs_sim = np.zeros((subs_size,subs_size))
for i in range(0,subs_size):
    print(i)
    a = subs['q_subjects'].iloc[i]
    na = subs['numBills'].iloc[i]
    for j in range(0,subs_size):
        if j < i:
            subs_sim[i,j] = subs_sim[j,i] #matrix is diagonal, so can cut loop time in half.
        else :
            b = subs['q_subjects'].iloc[j]
            nb = subs['numBills'].iloc[j]
            subs_sim[i,j]=jaccard_similarity(cur,a,b,na,nb)
cols = np.arange(subs_size)
rows = np.arange(subs_size)
ss_df = pd.DataFrame(data=subs_sim,index=rows,columns=cols)
sqlCommands.append_to_database(dbname,'subs_sim',subs_sim,engine)

subs_size =  691
0
1
2
